In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Vaihe 1: Määrittele Pydantic-mallit jäsennellyille tulosteille

Nämä mallit määrittelevät **skeeman**, jonka agentit palauttavat. `response_format`-ominaisuuden käyttö Pydanticin kanssa takaa:
- ✅ Tyyppiturvallinen datan poiminta
- ✅ Automaattinen validointi
- ✅ Ei virheitä vapaamuotoisten tekstivastausten jäsentämisessä
- ✅ Helppo ehdollinen reititys kenttien perusteella


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Vaihe 2: Luo hotellivaraustyökalu

Tämä työkalu on se, jota **availability_agent** käyttää tarkistaakseen, ovatko huoneet saatavilla. Käytämme `@ai_function`-koristetta seuraaviin tarkoituksiin:
- Muuntaa Python-funktion AI-kutsuttavaksi työkaluksi
- Luoda automaattisesti JSON-skeema LLM:lle
- Käsitellä parametrien validointi
- Mahdollistaa agenttien automaattisen kutsumisen

Tässä demossa:
- **Tukholma, Seattle, Tokio, Lontoo, Amsterdam** → Huoneita saatavilla ✅
- **Kaikki muut kaupungit** → Ei huoneita ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Vaihe 3: Määritä reitityksen ehtofunktiot

Nämä funktiot tarkistavat agentin vastauksen ja määrittävät, mitä polkua työnkulussa seurataan.

**Keskeinen toimintamalli:**
1. Tarkista, onko viesti `AgentExecutorResponse`
2. Jäsennä rakenteellinen tulos (Pydantic-malli)
3. Palauta `True` tai `False` reitityksen ohjaamiseksi

Työnkulku arvioi nämä ehdot **reunoilla** päättääkseen, mikä suorittaja kutsutaan seuraavaksi.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Vaihe 4: Luo mukautettu näyttöexecutor

Executorit ovat työnkulun komponentteja, jotka suorittavat muunnoksia tai sivuvaikutuksia. Käytämme `@executor`-koristetta luodaksemme mukautetun executorin, joka näyttää lopputuloksen.

**Keskeiset käsitteet:**
- `@executor(id="...")` - Rekisteröi funktion työnkulun executoriksi
- `WorkflowContext[Never, str]` - Tyyppivihjeet syötteelle/tulosteelle
- `ctx.yield_output(...)` - Tuottaa työnkulun lopullisen tuloksen


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Vaihe 5: Lataa ympäristömuuttujat

Määritä LLM-asiakas. Tämä esimerkki toimii seuraavien kanssa:
- **GitHub-mallit** (Ilmainen taso GitHub-tokenilla)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Vaihe 6: Luo tekoälyagentteja, joilla on jäsenneltyjä tuloksia

Luomme **kolme erikoistunutta agenttia**, joista jokainen on kääritty `AgentExecutor`-luokkaan:

1. **availability_agent** - Tarkistaa hotellien saatavuuden työkalun avulla
2. **alternative_agent** - Ehdottaa vaihtoehtoisia kaupunkeja (jos huoneita ei ole)
3. **booking_agent** - Kannustaa varaamaan (jos huoneita on saatavilla)

**Keskeiset ominaisuudet:**
- `tools=[hotel_booking]` - Tarjoaa työkalun agentille
- `response_format=PydanticModel` - Pakottaa jäsennellyn JSON-tulosteen
- `AgentExecutor(..., id="...")` - Käärii agentin työnkulun käyttöä varten


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Vaihe 7: Rakenna työnkulku ehdollisilla reiteillä

Nyt käytämme `WorkflowBuilder`-luokkaa graafin rakentamiseen ehdollisella reitityksellä:

**Työnkulun rakenne:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Keskeiset metodit:**
- `.set_start_executor(...)` - Määrittää aloituspisteen
- `.add_edge(from, to, condition=...)` - Lisää ehdollisen reitin
- `.build()` - Viimeistelee työnkulun


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Vaihe 8: Suorita testitapaus 1 - Kaupunki ILMAN saatavuutta (Pariisi)

Testataan **ei saatavuutta** -polkua pyytämällä hotelleja Pariisista (jossa simulaatiossamme ei ole huoneita).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Vaihe 9: Suorita testitapaus 2 - Kaupunki, jossa on saatavuutta (Tukholma)

Testataan nyt **saatavuus**-polkua pyytämällä hotelleja Tukholmasta (jossa on huoneita simulaatiossamme).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Keskeiset Opit ja Seuraavat Askeleet

### ✅ Mitä Olet Oppinut:

1. **WorkflowBuilder-malli**
   - Käytä `.set_start_executor()` määrittääksesi aloituspisteen
   - Käytä `.add_edge(from, to, condition=...)` ehdolliseen reititykseen
   - Kutsu `.build()` viimeistelläksesi työnkulun

2. **Ehdollinen Reititys**
   - Ehtofunktiot tarkistavat `AgentExecutorResponse`-vastaukset
   - Jäsennä rakenteelliset tulosteet reitityspäätösten tekemiseksi
   - Palauta `True` aktivoidaksesi reitin, `False` ohittaaksesi sen

3. **Työkalujen Integrointi**
   - Käytä `@ai_function` muuntaaksesi Python-funktiot AI-työkaluiksi
   - Agentit kutsuvat työkaluja automaattisesti tarpeen mukaan
   - Työkalut palauttavat JSON-muotoista dataa, jota agentit voivat jäsentää

4. **Rakenteelliset Tulosteet**
   - Käytä Pydantic-malleja tyyppiturvalliseen datan käsittelyyn
   - Aseta `response_format=MyModel` luodessasi agenteja
   - Jäsennä vastaukset `Model.model_validate_json()` avulla

5. **Mukautetut Suorittajat**
   - Käytä `@executor(id="...")` luodaksesi työnkulun komponentteja
   - Suorittajat voivat muuntaa dataa tai suorittaa sivuvaikutuksia
   - Käytä `ctx.yield_output()` tuottaaksesi työnkulun tuloksia

### 🚀 Käytännön Sovellukset:

- **Matkavaraukset**: Tarkista saatavuus, ehdota vaihtoehtoja, vertaa vaihtoehtoja
- **Asiakaspalvelu**: Reititä ongelmatyypin, tunteen tai prioriteetin perusteella
- **Verkkokauppa**: Tarkista varasto, ehdota vaihtoehtoja, käsittele tilauksia
- **Sisällön Moderointi**: Reititä toksisuusarvojen tai käyttäjän ilmoitusten perusteella
- **Hyväksymisprosessit**: Reititä summan, käyttäjäroolin tai riskitason perusteella
- **Monivaiheinen Käsittely**: Reititä datan laadun tai täydellisyyden perusteella

### 📚 Seuraavat Askeleet:

- Lisää monimutkaisempia ehtoja (useita kriteerejä)
- Toteuta silmukoita työnkulun tilanhallinnan avulla
- Lisää alatyönkulkuja uudelleenkäytettäviä komponentteja varten
- Integroi oikeisiin API:hin (hotellivaraus, varastojärjestelmät)
- Lisää virheenkäsittely ja varareitit
- Visualisoi työnkulut sisäänrakennetuilla visualisointityökaluilla



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
